En este script se realizan las predicciones de los modelos entrenados, sobre el conjunto de test.

Primero comenzamos importando las librerías necesarias

In [105]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import multilabel_confusion_matrix

Definimos las rutas necesarias

In [106]:
CWD = os.getcwd()
#CWD = os.path.dirname(__file__)
TEST_DATA_PATH = os.path.join(CWD, "dataset", "test")

# Ruta donde se guardaron los entrenamientos del paso anterior
RUNS_PATH = os.path.join(CWD, "runs")

# nombres de las clases ordenados alfabeticamente
CLASS_NAMES = sorted(os.listdir(TEST_DATA_PATH))
print(CLASS_NAMES)


# Ruta donde guardar matrices de confusión
TEST_RESULTS_PATH = os.path.join(CWD, "test_results")

if not os.path.exists(TEST_RESULTS_PATH):
    print(f"Creando directorio {TEST_RESULTS_PATH}.")
    os.mkdir(TEST_RESULTS_PATH)

# Nombres de las métricas que vamos a obtener de cada modelo
METRICS_NAMES = ["acc", "recall", "precision", "FNR", "FPR", "specificity", "F1", "AUC"]

['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']


A continuación, se define una función que calcula la predicciones del modelo $y\_pred$, y las devuelve junto con la clasificación real $y\_true$, y el $y\_score$

In [107]:
def make_predictions(model_path, test_path, class_list):
    """Compute predictions with the trained model and return y_true, y_pred, y_score"""

    model = YOLO(model_path)
    # print(model.names)

    y_true = []
    y_pred = []
    y_score = []
    
    # Iteramos sobre cada carpeta de clase (MEL, NV, etc.)
    for class_name in CLASS_NAMES:
        class_dir = os.path.join(test_path, class_name)
        if not os.path.isdir(class_dir): continue
            
        # Obtenemos el índice numérico de la clase (ej: MEL = 0)
        if class_name in class_list:
            class_idx = class_list.index(class_name)
        else:
            continue
            
        # Listamos imágenes
        images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]

        # Predecimos
        results = model.predict(images, verbose=False, imgsz=224)

        for res in results:
            # probs.top1 devuelve el índice de la clase con mayor probabilidad
            pred_idx = res.probs.top1
            y_score.append(res.probs.data.cpu().numpy())
            y_true.append(class_idx)
            y_pred.append(pred_idx)

    return np.array(y_true), np.array(y_pred), np.array(y_score)

Se define una función que genera la matríz de confusión y la guarda para poder ser utilizada más adelante

In [108]:
def get_confusion_matrix(y_true, y_pred, model_name):
    """Function to generate the confusion matrix of the model given its predictions"""
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)

    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, 
        display_labels=CLASS_NAMES
    )
    disp.plot()

    # Guardamos la matriz de confunsión en una imagen (p.ej "CM_yolo11n.png")
    plt.savefig(os.path.join(TEST_RESULTS_PATH, f"CM_{model_name}.png"))

    plt.close()

Se define un método que genera la curva ROC, la guarda y la devuelve para poder ser utilizada más adelante

In [109]:
def get_ROC_curve_and_AUC(y_true, y_score, model_name):
    """
    Function to generate the ROC curve of the model using the "One vs Rest" approach and
    return the mean of the AUC values for each positive class
    """

    fig, ax = plt.subplots()

    for class_id in range(len(CLASS_NAMES)):
        fpr, tpr, _ = roc_curve(y_true, y_score[:, class_id], pos_label=class_id)  
        ax.plot(
            fpr, tpr, 
            label=f"ROC curve - Positive class: {CLASS_NAMES[class_id]}"
        )

    ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Curva ROC One-vs-Rest: {model_name}"
    )
    
    ax.legend()
    fig.savefig(os.path.join(TEST_RESULTS_PATH, f"ROC_{model_name}.png")) 

    plt.close(fig)

    return np.mean(roc_auc_score(y_true, y_score, average=None, multi_class="ovr"))

Se define una función que calcula y devuelve las métricas de rendimiento del modelo el la fase de predicción 

In [110]:
def compute_metrics(y_true, y_pred):
    """Function to compute all the necessary metrics using the "One vs Rest" approach"""
    mcm = multilabel_confusion_matrix(y_true, y_pred)
    
    tn = mcm[:, 0, 0]
    tp = mcm[:, 1, 1]
    fn = mcm[:, 1, 0]
    fp = mcm[:, 0, 1]

    accuracy = accuracy_score(y_true, y_pred)
    recall = np.mean(tp / (tp + fn))
    precision = np.mean(tp / (tp + fp))
    fnr = np.mean(fn / (fn + tp))
    fpr = np.mean(fp / (fp + tn))
    specificity = 1 - fpr
    f1 = 2 * precision * recall / (precision + recall)

    return accuracy, recall, precision, fnr, fpr, specificity, f1

Para cada modelo, se va a calcular su rendimiento en base a sus predicciones. El rendimiento de cada modelo será guardado para poder ser comparado más adelante.

In [111]:
metrics = {}
models = ["yolo11n", "yolo11s", "yolo11m"]

for variant in models:
    # Construimos path al archivo best.pt
    model_path = os.path.join(RUNS_PATH, f'train_{variant}-cls.pt', 'weights', 'best.pt')
    
    if not os.path.exists(model_path):
        print(f"No se encontró el modelo {variant} en {model_path}")
        continue
        
    print(f"\n-------------------Evaluando {variant}-------------------")
    print(f"Ruta del modelo: {model_path}")

    # Obtener las predicciones para el conjunto de test
    y_true, y_pred, y_score = make_predictions(model_path, TEST_DATA_PATH, CLASS_NAMES)

    # print(y_true)
    # print(y_pred)


    # Matriz de confusión del modelo
    get_confusion_matrix(y_true, y_pred, model_name=variant)
    
    # Calculamos todas las métricas de rendimiento necesarias
    acc, recall, precision, fnr, fpr, specificity, f1 = \
        compute_metrics(y_true=y_true, y_pred=y_pred)
    
    # Obtenemos la curva ROC
    auc = get_ROC_curve_and_AUC(y_true, y_score, model_name=variant)

    # Guardamos las métricas
    metrics[variant] = [acc, recall, precision, fnr, fpr, specificity, f1, auc]

    
# Creamos un dataframe con todas las métricas
df_metrics = pd.DataFrame(
    metrics,
    index=METRICS_NAMES
).T

# Convertimos el dataframe a latex y a csv
df_metrics.to_latex(
    os.path.join(TEST_RESULTS_PATH, "metrics.tex"),
    caption="Rendimiento de las tres versiones de YOLO"
)

df_metrics.to_csv(os.path.join(TEST_RESULTS_PATH, "metrics.csv"))

print("\nTodas las métricas se han calculado y guardado correctamente.")



-------------------Evaluando yolo11n-------------------
Ruta del modelo: c:\Users\gonza\Desktop\universidad\Malaga\Tercero\Aprendizaje_Automatico\practicas-aprendizaje\practica4\runs\train_yolo11n-cls.pt\weights\best.pt

-------------------Evaluando yolo11s-------------------
Ruta del modelo: c:\Users\gonza\Desktop\universidad\Malaga\Tercero\Aprendizaje_Automatico\practicas-aprendizaje\practica4\runs\train_yolo11s-cls.pt\weights\best.pt

-------------------Evaluando yolo11m-------------------
Ruta del modelo: c:\Users\gonza\Desktop\universidad\Malaga\Tercero\Aprendizaje_Automatico\practicas-aprendizaje\practica4\runs\train_yolo11m-cls.pt\weights\best.pt

Todas las métricas se han calculado y guardado correctamente.
